# How regional are the peers on each IXP?
<pre>
 Considering each IXP participant on IXP open peering and the region each AS is registered 
 How regional are the neighbors?
</pre>


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly 
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import HTML
from IPython.display import display
import os
import re
import glob
import fnmatch
from datetime import date, datetime

import matplotlib.pyplot as plt
import seaborn as sns
import timeit

from pandarallel import pandarallel
import argparse
import sys

# Globals

In [2]:
# Download routing tables to this folder
BGP_DATA  = "/Users/bertholdo/tmp/Routing_Tables/"      
# Output images 
IMAGE_DIR = "/Users/bertholdo/PAPERS/asym/leandro/"
# in/out datasets
DATASET   = "/Users/bertholdo/ut_gitlab/asy_pathsize/dataset/"

version='1.0'

# Functions

In [3]:
def is_jupyter_notebook():
    ''' Return True if is running inside Jupyter Notebook or Jupyter lab '''
    try:
        __IPYTHON__
        _in_ipython_session = True
    except NameError:
        _in_ipython_session = False
    return _in_ipython_session

In [4]:
import shutil
from urllib import request
from contextlib import closing

def download_url(url='',dstDir=''):   
    # if url is abc/xyz/file.txt, the dstFile will be file.txt
    dstFile = dstDir + url.rsplit('/',1)[1]
        
    #print("downloading FROM:",url, "TO", dstFile)
    with closing(request.urlopen(url)) as src:
        with open(dstFile, 'wb') as dst:
            shutil.copyfileobj(src, dst)
            
    return dstFile;

In [5]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
from functools import partial
import os


def download_rir_files(rir_dir='./Dados_RIR/'):
    rir_url =['ftp://ftp.lacnic.net:/pub/stats/lacnic/delegated-lacnic-extended-latest',
              'ftp://ftp.afrinic.net:/pub/stats/afrinic/delegated-afrinic-extended-latest',
              'ftp://ftp.arin.net:/pub/stats/arin/delegated-arin-extended-latest',
              'ftp://ftp.apnic.net:/public/apnic/stats/apnic/delegated-apnic-extended-latest',
              'ftp://ftp.ripe.net:/pub/stats/ripencc/delegated-ripencc-extended-latest']
     
    if not os.path.exists(rir_dir):
        os.makedirs(rir_dir)
    
    # simple download
    #for url in rir_url:
    #    file = rir_dir + url.rsplit('/',1)[1]
    #    download_url(url,file)
 
    # paralled download
    # Prepare one param to ThreadPool (partial to join parms)
    call_funct_param=partial(download_url, dstDir=rir_dir)
  
    # Start Thread for download URL
    results = ThreadPool(8).imap_unordered(call_funct_param, rir_url)
    
    # Wait to finish
    for path in results:
        print("Downloaded to: ", path)
              
    return;

In [6]:
def rir_description( descr ):
    if 'RIPE' in descr:
        return 'ripencc'
    elif 'APNIC' in descr:
        return 'apnic'
    elif 'AFRINIC' in descr:
        return 'afrinic'
    elif 'LACNIC' in descr:
        return 'lacnic'
    elif 'ARIN' in descr:
        return 'arin'
    elif 'Reserved' in descr or 'AS_TRANS' in descr:
        return 'Reserved'
    else:
        print ('RIR_DESCRIPTION:: Not know -->', descr)

def rir_iana_delegation():
    '''Return a dataframe from all ASNs alocated to each RIR from IANA'''   
    
    asn16bits="https://www.iana.org/assignments/as-numbers/as-numbers-1.csv"
    asn32bits="https://www.iana.org/assignments/as-numbers/as-numbers-2.csv"
    mydict={}

    for url in [asn16bits, asn32bits]:
        #print (url)
        iana=pd.read_csv(url)
        for index,row in iana.iterrows():
            if any (x in row['Description'] for x in ['16-bit AS numbers','Unallocated','Reserved','AS_TRANS']):
                continue
            if '-' in row['Number']:
                start,end=[int(n) for n in row['Number'].split('-')]
                for n in range (start,end):
                    if not n in mydict.keys():
                        mydict[n]=rir_description(row['Description'])
                    else:
                        print('tem-->', n)
            else:
                n=int(row['Number'])
                if not n in mydict.keys():
                    mydict[n]=rir_description(row['Description'])
                else:
                    print('ja tem')
                    
    df=pd.DataFrame.from_dict(mydict, orient='index', columns=['RIR'])
    df['ASN'] = df.index
    df=df.reset_index()
    
    return (df[['ASN','RIR']])

def rir_asn_delegation():
    ''' Read ASN info from RIR files in a dataframe pandas'''

    # Download last version of rir data
    data_RIR='./Dados_RIR/'
    download_rir_files(data_RIR)

    # Read all RIR files
    rir_df=pd.DataFrame()
    for file in os.listdir(data_RIR):
        print ("Appending..", file)
        rir_database_path=data_RIR+file
        #print (rir_database_path)
        headers = ['Registry', 'Country Code', 'Type', 'Start', 'Value', 'Date', 'Status', 'Extensions']
        tmp_df = pd.read_csv(rir_database_path, delimiter='|', comment='#', names=headers, dtype=str, 
                             keep_default_na=False, na_values=[''],encoding='utf-8')[4:]
        #rir_df=rir_df.append(tmp_df)
        rir_df=pd.concat([rir_df,tmp_df])
    
    #filter rir_df fields  (asn,rir)
    df=rir_df[rir_df['Type']=='asn']
    df=df.filter(['Start','Registry'])
    df=df.drop_duplicates()
    df.columns = ['ASN','RIR']
    df["ASN"]=pd.to_numeric(df["ASN"])
    
    return (df.reset_index(drop=True))

def asn_allocation_rir_iana():
    ''' Get all ASNs information about region, including IANA reserve for RIRs'''
    df1=rir_asn_delegation()
    df2=rir_iana_delegation()

    df=pd.concat([df1,df2]).drop_duplicates().reset_index(drop=True)
    
    return df


# Main

In [7]:
df_rir=asn_allocation_rir_iana()

Downloaded to:  ./Dados_RIR/delegated-arin-extended-latest
Downloaded to:  ./Dados_RIR/delegated-lacnic-extended-latest
Downloaded to:  ./Dados_RIR/delegated-afrinic-extended-latest
Downloaded to:  ./Dados_RIR/delegated-ripencc-extended-latest
Downloaded to:  ./Dados_RIR/delegated-apnic-extended-latest
Appending.. delegated-afrinic-extended-latest
Appending.. delegated-arin-extended-latest
Appending.. delegated-apnic-extended-latest
Appending.. delegated-ripencc-extended-latest
Appending.. delegated-lacnic-extended-latest


In [8]:
ixp_list=['amsix','spoixbr','six','linx','poaixbr']

for ixp in ixp_list:
    for f in glob.glob(f'peerinfo/*_{ixp}.csv.gz'):
        df = pd.read_csv(f)
        #df = df_asnlst[(df_asnlst['policy'].str.upper()=='OPEN') & (df_asnlst['state'].str.upper()=='UP')]
        #ixp_asn_lst = df_asnlst['ASN'].unique().tolist()
        #df=df[df['as_neigh'].isin(ixp_asn_lst)]
    
        #df=pd.read_csv(f'peerinfo/2022-05-05_peerinfo_br-gru_spoixbr.csv.gz')
        df=df[(df['policy'].str.upper()=='OPEN') & (df['state'].str.upper()=='UP')]
        df=df[['ASN','organization']]
        df_peer=df.drop_duplicates()
    
        # Where are from each peer?
        nei_list = df_peer['ASN'].to_list()
        df=df_rir[df_rir['ASN'].isin(nei_list)]
        
        # Contabil
        print(f'==== Values from [{ixp}] ====')
        display(df['RIR'].value_counts(ascending=False, normalize=True))
        print('\n')

==== Values from [amsix] ====


ripencc    0.745935
arin       0.134146
apnic      0.083333
afrinic    0.018293
lacnic     0.018293
Name: RIR, dtype: float64



==== Values from [spoixbr] ====


lacnic     0.927386
arin       0.055498
ripencc    0.012448
apnic      0.003631
afrinic    0.001037
Name: RIR, dtype: float64



==== Values from [six] ====


arin       0.849515
ripencc    0.092233
apnic      0.058252
Name: RIR, dtype: float64



==== Values from [linx] ====


ripencc    0.734783
arin       0.123913
apnic      0.080435
afrinic    0.058696
lacnic     0.002174
Name: RIR, dtype: float64



==== Values from [poaixbr] ====


lacnic     0.914530
arin       0.081197
ripencc    0.004274
Name: RIR, dtype: float64